In [184]:
#Michelle Brenes
#Laser Rivera
#Julio Marin 
#Mohnish Hemdev
#Jayson Morán

In [185]:
#pip install pandas
#pip install dash
#pip install geopandas
#pip install pipwin
#pip install fiona
#pip install wheel

In [186]:
import dash
import dash_core_components as dcc 
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input,Output
import matplotlib.pyplot as plt
from urllib.request import urlopen
import geopandas as gpd

In [187]:
#Importe del json que contiene al mapa de panama
path = 'https://raw.githubusercontent.com/JaysonAMR/Semestral_Topicos/main/Json/PAN_ADM01_PROV_V01.topo.json'
geo_pan = gpd.read_file(path)

#Importe del csv con los datos del movimiento de contenedores de los puertos
df = pd.read_csv('https://raw.githubusercontent.com/JaysonAMR/Semestral_Topicos/main/Resumido-por-a%C3%B1os.csv')

In [188]:
geo_pan.head() #para ver lo que tiene el json de panamá

,id,COD_PROV,PROV_NAME,ISO_CODE,geometry
0,None,01,Bocas del Toro,PA-1,"MULTIPOLYGON (((-82.00026 8.95718, -82.01260 8..."
1,None,04,Chiriquí,PA-4,"MULTIPOLYGON (((-82.44513 7.87112, -82.44337 7..."
2,None,02,Coclé,PA-2,"POLYGON ((-80.13616 8.96596, -80.14497 8.94522..."
3,None,03,Colón,PA-3,"MULTIPOLYGON (((-79.85114 9.39667, -79.84644 9..."
4,None,05,Darién,PA-5,"MULTIPOLYGON (((-78.01407 8.23574, -77.99350 8..."


In [189]:
df.head() #para ver lo que tiene el data frame

,COD_PROV,PROV_NAME,Puerto,Año,Trasbordo,Local,Zona Libre
0,1,Bocas del Toro,Bocas Fruit Co.,2017,0,16488,0
1,1,Bocas del Toro,Bocas Fruit Co.,2018,0,22896,0
2,1,Bocas del Toro,Bocas Fruit Co.,2019,0,58534,0
3,1,Bocas del Toro,Bocas Fruit Co.,2020,12362,83400,0
4,1,Bocas del Toro,Bocas Fruit Co.,2021,53958,63834,0


In [190]:
app = dash.Dash(__name__) #creacion del dash

In [191]:

app.layout = html.Div([
    
    html.Div([ #Creacion del Titulo
        html.H1('Movimientos de contenedores en el SPN'), #nombre
        html.Img(src='assets/contenedor.png') #introduccion de figura de contenedores
    ], className = 'banner'),#assets perzonalizado de color azul

    html.Div([
        html.Div([#creacion de los radio boton
            html.P('Selecciona el tipo de transito', className= 'flix_label', style={'color':'black', 'margin-top': '2px'}),
            dcc.RadioItems( id= 'tipo-radioitem',
                            labelStyle={'display': 'inline-block'},
                            options=[
                                {'label' : 'Local', 'value' : 'Local'},
                                {'label' : 'Trasbordo', 'value' : 'Trasbordo'},
                                {'label' : 'Zona Libre', 'value' : 'Zona Libre'}
                            ], value= 'Local',
                            style= {'text-aling' : 'center' , 'color' : 'black'}, className= 'dcc_compon'),
        ], className= 'create_container2 five columns', style= {'margin-bottom': '20px'}), #assets personalizado para tener un fondo rectangular
        
    ], className = 'row flex-display'),

    html.Div([
        html.Div([#Creacion del espacio para la grafica 1, grafica de la izquierda
            dcc.Graph(id = 'my_graph', figure = {})
        ], className= 'create_container2 nine colums'), 

        html.Div([#Creacion del espacio para la grafica 2, mapa de la derecha
            dcc.Graph(id = 'mpie_graph', figure = {})
        ], className= 'create_container2 nine colums'), 
    ], className='row flex-display'),

],id='mainContainer', style= {'display': 'flex', 'flex-direction': 'column'})#assets perzonalizados para tener un fondo rectangular

In [192]:
#GRAFICA
@app.callback (
    Output('my_graph', component_property= 'figure'), #2. para imprimirlo en el espacio seleccionado 
    [Input('tipo-radioitem', component_property='value')]#1. se toca el radio boton, ejecuta lo que está abajo y retorna fi y llama a my_grap
)

def update_graph(value):
    if value == 'Local':
        fig = px.bar(
            data_frame=df,
            animation_frame='Año', #No se como ponerlo arriba
            x = 'Puerto',
            y = 'Local'
        )
    elif value == 'Trasbordo':
        fig = px.bar(
            data_frame=df,
            animation_frame='Año', #No se como ponerlo arriba
            x = 'Puerto',
            y = 'Trasbordo'
        )
    else:
        fig = px.bar(
            data_frame=df,
            animation_frame='Año', #No se como ponerlo arriba
            x = 'Puerto',
            y = 'Zona Libre'
        )
    return fig

#MAPA

@app.callback (
    Output('mpie_graph', component_property= 'figure'),#2. espacio seleccionado apra imprimir el mapa
    [Input('tipo-radioitem', component_property='value')]#1. Ejecuta la parte de abajo y retorna fig para imprimirla en mpie_grapgh
)
def update_graph(value):
    if value == 'Local':
        fig = px.choropleth(data_frame=df, 
                    geojson=geo_pan, 
                    locations='PROV_NAME', # nombre de la columna del Dataframe
                    featureidkey='properties.PROV_NAME',  # ruta al campo del archivo GeoJSON con el que se hará la relación (nombre de los estados)
                    color='Local', #El color depende de las cantidades
                    color_continuous_scale="greens", #greens
                    animation_frame='Año' #barra de animacion de año
                    #scope="north america"
                   )
        fig.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
    elif value == 'Trasbordo':
        fig = px.choropleth(data_frame=df, 
                    geojson=geo_pan, 
                    locations='PROV_NAME', # nombre de la columna del Dataframe
                    featureidkey='properties.PROV_NAME',  # ruta al campo del archivo GeoJSON con el que se hará la relación (nombre de los estados)
                    color='Trasbordo', #El color depende de las cantidades
                    color_continuous_scale="greens", #greens
                    animation_frame='Año' #barra de animacion de año
                    #scope="north america"
                   )
        fig.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
    else:
        fig = px.choropleth(data_frame=df, 
                    geojson=geo_pan, 
                    locations='PROV_NAME', # nombre de la columna del Dataframe
                    featureidkey='properties.PROV_NAME',  # ruta al campo del archivo GeoJSON con el que se hará la relación (nombre de los estados)
                    color='Zona Libre', #El color depende de las cantidades
                    color_continuous_scale="greens", #greens
                    animation_frame='Año' #barra de animacion de año
                    #scope="north america"
                   )
        fig.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
    return fig
    

In [193]:
if __name__ == ('__main__'): #para iniciar el app
    app.run_server(port=5003)
#df = pd.read_csv("D:\CATALAG\Archivos Semestral\PSFC.csv")
#df.head()

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5003/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jul/2022 13:30:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 13:30:42] "GET /assets/s1.css?m=1627386984.0 HTTP/1.1" 304 -
127.0.0.1 - - [17/Jul/2022 13:30:42] "GET /assets/stylesheet.css?m=1657734382.2705944 HTTP/1.1" 304 -
127.0.0.1 - - [17/Jul/2022 13:30:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 13:30:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 13:30:42] "GET /_favicon.ico?v=2.5.1 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 13:30:42] "GET /assets/contenedor.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Jul/2022 13:30:42] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 13:30:42] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 13:30:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 13:30:43] "POST /_dash-update-component HTTP/1.1" 200 